# Day 12: Context API

## **Learning Objectives**
By the end of this lesson, students will be able to:
- Understand the prop drilling problem
- Create and use Context in React
- Use the useContext Hook
- Build a global state management system
- Share data across multiple components without prop drilling

---

## **Part 1: The Prop Drilling Problem**

### **What is Prop Drilling?**

Prop drilling is when you pass props through multiple levels of components, even when intermediate components don't need the data.

**Example of Prop Drilling:**

```javascript
// App needs to pass user to Profile, which passes to UserCard, which passes to Avatar
function App() {
  const user = { name: 'Chidi', avatar: 'photo.jpg' };
  
  return <Profile user={user} />;
}

function Profile({ user }) {
  // Profile doesn't use user, just passes it down
  return <UserCard user={user} />;
}

function UserCard({ user }) {
  // UserCard doesn't use user, just passes it down
  return <Avatar user={user} />;
}

function Avatar({ user }) {
  // Finally, Avatar uses user
  return <img src={user.avatar} alt={user.name} />;
}
```

**Problems:**
- Tedious to pass props through many levels
- Hard to maintain
- Components become tightly coupled
- Intermediate components receive props they don't use

---


## **Part 2: Introduction to Context API**

### **What is Context?**

Context provides a way to share data across the entire component tree without passing props manually at every level.

**Think of it like:**
- **Without Context:** Passing a message person-to-person through a chain
- **With Context:** Broadcasting a message everyone can hear

### **When to Use Context**

✅ **Good for:**
- Theme (dark/light mode)
- User authentication
- Language settings
- Global app settings

❌ **Not good for:**
- Frequently changing data
- Props that only go 1-2 levels down
- Everything (overusing context)

---


## **Part 3: Creating Context**

### **Step 1: Create Context**

```javascript
import { createContext } from 'react';

// Create context with default value
const UserContext = createContext(null);

export default UserContext;
```

### **Step 2: Provide Context**

Wrap components that need access to the context:

```javascript
import { useState } from 'react';
import UserContext from './UserContext';

function App() {
  const [user, setUser] = useState({ name: 'Chidi', role: 'Developer' });
  
  return (
    <UserContext.Provider value={user}>
      <Profile />
      <Dashboard />
    </UserContext.Provider>
  );
}
```

### **Step 3: Consume Context**

Use the `useContext` Hook to access context value:

```javascript
import { useContext } from 'react';
import UserContext from './UserContext';

function Profile() {
  const user = useContext(UserContext);
  
  return (
    <div>
      <h2>{user.name}</h2>
      <p>{user.role}</p>
    </div>
  );
}
```

---


## **Part 4: Complete Context Pattern**

### **Better Organization**

Create a custom context file with provider:

**contexts/UserContext.jsx:**

```javascript
import { createContext, useState, useContext } from 'react';

// Create context
const UserContext = createContext();

// Create provider component
export function UserProvider({ children }) {
  const [user, setUser] = useState({
    name: 'Chidi Okafor',
    email: 'chidi@example.com',
    role: 'Developer'
  });
  
  function updateUser(newUser) {
    setUser(newUser);
  }
  
  return (
    <UserContext.Provider value={{ user, updateUser }}>
      {children}
    </UserContext.Provider>
  );
}

// Create custom hook for easy access
export function useUser() {
  const context = useContext(UserContext);
  if (!context) {
    throw new Error('useUser must be used within UserProvider');
  }
  return context;
}

export default UserContext;
```

**Using it in App:**

```javascript
import { UserProvider } from './contexts/UserContext';
import Profile from './components/Profile';
import Dashboard from './components/Dashboard';

function App() {
  return (
    <UserProvider>
      <Profile />
      <Dashboard />
    </UserProvider>
  );
}
```

**Using it in components:**

**components/Profile.jsx:**
```javascript
import { useUser } from '../contexts/UserContext';

function Profile() {
  const { user, updateUser } = useUser();
  
  function handleUpdate() {
    updateUser({ ...user, name: 'New Name' });
  }
  
  return (
    <div>
      <h2>{user.name}</h2>
      <p>{user.email}</p>
      <button onClick={handleUpdate}>Update Name</button>
    </div>
  );
}
```

**components/Dashboard.jsx:**
```javascript
import { useUser } from '../contexts/UserContext';

function Dashboard() {
  const { user } = useUser();

  return (
    <div style={{ border: '1px solid #ccc', padding: '10px' }}>
      <h2>Dashboard</h2>
      <p>Welcome back, {user.name}!</p>
      <p>Your role: {user.role}</p>
    </div>
  );
}
```

---


## **Part 5: Multiple Contexts**

You can use multiple contexts in the same app:

```javascript
import { UserProvider } from './contexts/UserContext';
import { ThemeProvider } from './contexts/ThemeContext';

function App() {
  return (
    <UserProvider>
      <ThemeProvider>
        <Dashboard />
      </ThemeProvider>
    </UserProvider>
  );
}
```

**In components:**

```javascript
import { useUser } from '../contexts/UserContext';
import { useTheme } from '../contexts/ThemeContext';

function Dashboard() {
  const { user } = useUser();
  const { theme, toggleTheme } = useTheme();
  
  return (
    <div style={{ background: theme === 'dark' ? '#333' : '#fff' }}>
      <h1>Welcome, {user.name}</h1>
      <button onClick={toggleTheme}>Toggle Theme</button>
    </div>
  );
}
```

---


## **Part 6: Complete Example - Nigerian E-Commerce App with Cart**

Let's build a shopping cart system using Context API.

### **contexts/CartContext.jsx**

```javascript
import { createContext, useState, useContext } from 'react';

const CartContext = createContext();

export function CartProvider({ children }) {
  const [cartItems, setCartItems] = useState([]);
  
  function addToCart(product) {
    const existingItem = cartItems.find(item => item.id === product.id);
    
    if (existingItem) {
      setCartItems(cartItems.map(item =>
        item.id === product.id
          ? { ...item, quantity: item.quantity + 1 }
          : item
      ));
    } else {
      setCartItems([...cartItems, { ...product, quantity: 1 }]);
    }
  }
  
  function removeFromCart(productId) {
    setCartItems(cartItems.filter(item => item.id !== productId));
  }
  
  function updateQuantity(productId, quantity) {
    if (quantity === 0) {
      removeFromCart(productId);
    } else {
      setCartItems(cartItems.map(item =>
        item.id === productId ? { ...item, quantity } : item
      ));
    }
  }
  
  function clearCart() {
    setCartItems([]);
  }
  
  const totalItems = cartItems.reduce((sum, item) => sum + item.quantity, 0);
  const totalPrice = cartItems.reduce((sum, item) => sum + (item.price * item.quantity), 0);
  
  return (
    <CartContext.Provider value={{
      cartItems,
      addToCart,
      removeFromCart,
      updateQuantity,
      clearCart,
      totalItems,
      totalPrice
    }}>
      {children}
    </CartContext.Provider>
  );
}

export function useCart() {
  const context = useContext(CartContext);
  if (!context) {
    throw new Error('useCart must be used within CartProvider');
  }
  return context;
}
```

### **App.jsx**

```javascript
import { CartProvider } from './contexts/CartContext';
import Navbar from './components/Navbar';
import ProductList from './components/ProductList';
import Cart from './components/Cart';
import './App.css';

function App() {
  return (
    <CartProvider>
      <div className="app">
        <Navbar />
        <div className="container">
          <ProductList />
          <Cart />
        </div>
      </div>
    </CartProvider>
  );
}

export default App;
```

### **components/Navbar.jsx**

```javascript
import { useCart } from '../contexts/CartContext';
import './Navbar.css';

function Navbar() {
  const { totalItems } = useCart();
  
  return (
    <nav className="navbar">
      <h1>🇳🇬 Naija Store</h1>
      <div className="cart-badge">
        🛒 Cart ({totalItems})
      </div>
    </nav>
  );
}

export default Navbar;
```

### **components/ProductList.jsx**

```javascript
import { useCart } from '../contexts/CartContext';
import './ProductList.css';

function ProductList() {
  const { addToCart } = useCart();
  
  const products = [
    { id: 1, name: 'Jollof Rice', price: 1500, image: '🍚' },
    { id: 2, name: 'Suya', price: 1000, image: '🍖' },
    { id: 3, name: 'Chin Chin', price: 500, image: '🍪' },
    { id: 4, name: 'Puff Puff', price: 300, image: '🍩' },
    { id: 5, name: 'Akara', price: 200, image: '🥟' },
    { id: 6, name: 'Moi Moi', price: 400, image: '🫘' }
  ];
  
  return (
    <div className="product-list">
      <h2>Products</h2>
      <div className="products-grid">
        {products.map(product => (
          <div key={product.id} className="product-card">
            <div className="product-image">{product.image}</div>
            <h3>{product.name}</h3>
            <p className="price">₦{product.price.toLocaleString()}</p>
            <button onClick={() => addToCart(product)} className="add-btn">
              Add to Cart
            </button>
          </div>
        ))}
      </div>
    </div>
  );
}

export default ProductList;
```

### **components/Cart.jsx**

```javascript
import { useCart } from '../contexts/CartContext';
import './Cart.css';

function Cart() {
  const { cartItems, removeFromCart, updateQuantity, clearCart, totalPrice } = useCart();
  
  if (cartItems.length === 0) {
    return (
      <div className="cart">
        <h2>Shopping Cart</h2>
        <p className="empty-cart">Your cart is empty</p>
      </div>
    );
  }
  
  return (
    <div className="cart">
      <h2>Shopping Cart</h2>
      
      <div className="cart-items">
        {cartItems.map(item => (
          <div key={item.id} className="cart-item">
            <div className="item-info">
              <span className="item-image">{item.image}</span>
              <div>
                <h4>{item.name}</h4>
                <p>₦{item.price.toLocaleString()}</p>
              </div>
            </div>
            
            <div className="item-controls">
              <button onClick={() => updateQuantity(item.id, item.quantity - 1)}>
                -
              </button>
              <span>{item.quantity}</span>
              <button onClick={() => updateQuantity(item.id, item.quantity + 1)}>
                +
              </button>
              <button 
                onClick={() => removeFromCart(item.id)}
                className="remove-btn"
              >
                Remove
              </button>
            </div>
          </div>
        ))}
      </div>
      
      <div className="cart-footer">
        <h3>Total: ₦{totalPrice.toLocaleString()}</h3>
        <button onClick={clearCart} className="clear-btn">
          Clear Cart
        </button>
        <button className="checkout-btn">
          Checkout
        </button>
      </div>
    </div>
  );
}

export default Cart;
```

### **CSS Files**

**App.css:**
```css
.app {
  min-height: 100vh;
  background: #f5f5f5;
}

.container {
  max-width: 1400px;
  margin: 0 auto;
  padding: 2rem;
  display: grid;
  grid-template-columns: 2fr 1fr;
  gap: 2rem;
}

@media (max-width: 768px) {
  .container {
    grid-template-columns: 1fr;
  }
}
```

**Navbar.css:**
```css
.navbar {
  background: #008751;
  color: white;
  padding: 1rem 2rem;
  display: flex;
  justify-content: space-between;
  align-items: center;
  box-shadow: 0 2px 5px rgba(0,0,0,0.1);
}

.navbar h1 {
  margin: 0;
}

.cart-badge {
  background: rgba(255,255,255,0.2);
  padding: 0.5rem 1rem;
  border-radius: 20px;
  font-weight: bold;
}
```

**ProductList.css:**
```css
.product-list h2 {
  color: #333;
  margin-bottom: 1.5rem;
}

.products-grid {
  display: grid;
  grid-template-columns: repeat(auto-fill, minmax(200px, 1fr));
  gap: 1.5rem;
}

.product-card {
  background: white;
  padding: 1.5rem;
  border-radius: 10px;
  text-align: center;
  box-shadow: 0 2px 8px rgba(0,0,0,0.1);
  transition: transform 0.3s;
}

.product-card:hover {
  transform: translateY(-5px);
}

.product-image {
  font-size: 4rem;
  margin-bottom: 1rem;
}

.product-card h3 {
  margin: 0.5rem 0;
  color: #333;
}

.price {
  color: #008751;
  font-weight: bold;
  font-size: 1.2rem;
  margin: 0.5rem 0;
}

.add-btn {
  width: 100%;
  padding: 0.8rem;
  background: #008751;
  color: white;
  border: none;
  border-radius: 5px;
  cursor: pointer;
  font-size: 1rem;
  transition: background 0.3s;
}

.add-btn:hover {
  background: #006741;
}
```

**Cart.css:**
```css
.cart {
  background: white;
  padding: 1.5rem;
  border-radius: 10px;
  box-shadow: 0 2px 8px rgba(0,0,0,0.1);
  position: sticky;
  top: 2rem;
  max-height: 600px;
  overflow-y: auto;
}

.cart h2 {
  color: #333;
  margin-bottom: 1.5rem;
}

.empty-cart {
  text-align: center;
  color: #999;
  padding: 2rem;
}

.cart-items {
  margin-bottom: 1.5rem;
}

.cart-item {
  border-bottom: 1px solid #eee;
  padding: 1rem 0;
}

.item-info {
  display: flex;
  gap: 1rem;
  margin-bottom: 0.5rem;
}

.item-image {
  font-size: 2rem;
}

.item-info h4 {
  margin: 0;
  color: #333;
}

.item-info p {
  margin: 0.2rem 0;
  color: #666;
}

.item-controls {
  display: flex;
  gap: 0.5rem;
  align-items: center;
}

.item-controls button {
  padding: 0.3rem 0.8rem;
  border: 1px solid #ddd;
  background: white;
  cursor: pointer;
  border-radius: 3px;
}

.item-controls button:hover {
  background: #f5f5f5;
}

.remove-btn {
  background: #d32f2f !important;
  color: white !important;
  border: none !important;
  margin-left: auto;
}

.cart-footer {
  border-top: 2px solid #eee;
  padding-top: 1rem;
}

.cart-footer h3 {
  color: #008751;
  margin-bottom: 1rem;
}

.clear-btn, .checkout-btn {
  width: 100%;
  padding: 0.8rem;
  border: none;
  border-radius: 5px;
  cursor: pointer;
  font-size: 1rem;
  margin-top: 0.5rem;
}

.clear-btn {
  background: #f5f5f5;
  color: #333;
}

.checkout-btn {
  background: #008751;
  color: white;
}

.checkout-btn:hover {
  background: #006741;
}
```

### **What We Used:**
✅ **Context API** - Global cart state  
✅ **useContext Hook** - Accessing cart from any component  
✅ **Custom Hook** - useCart for easy access  
✅ **Provider Pattern** - CartProvider wrapping app  
✅ **State Management** - Add, remove, update cart items  
✅ **Calculations** - Total items and price  
✅ **Nigerian Context** - Local foods and Naira currency  

---


## **Part 7: Tasks**

## 🎯 Task 1: Theme Switcher

**Task:** Create a dark/light theme toggle using Context

**Requirements:**
1. Create `ThemeContext.jsx`
2. Provide theme state (light/dark)
3. Create `toggleTheme` function
4. Use context in Navbar to show current theme
5. Use context in multiple components to apply theme
6. Toggle button switches between themes

**Sample Output:**
```
Current Theme: Dark 🌙
[Toggle Theme]

(All components change background/text colors)
```

**Challenge:** Save theme preference to localStorage

---

## 🎯 Task 2: User Authentication Context

**Task:** Build a simple authentication system with Context

**Requirements:**
1. Create `AuthContext.jsx`
2. State: user (null or user object), isLoggedIn
3. Functions: login(username, password), logout()
4. Login form that uses login function
5. Show different UI based on isLoggedIn
6. Display username when logged in

**Sample Output:**
```
Not logged in:
[Login] button

Logged in:
Welcome, Chidi! [Logout]
```

**Challenge:** Add "Protected Routes" that require login

---

## 🎯 Task 3: Language Selector

**Task:** Create a multi-language app using Context

**Requirements:**
1. Create `LanguageContext.jsx`
2. Support: English, Yoruba, Igbo, Hausa
3. Store current language in context
4. Create translations object for common phrases
5. Language selector dropdown
6. Use context to display translated text

**Sample Output:**
```
Language: [English ▼]

Welcome to our app!
---
Language: [Yoruba ▼]

Ẹ káàbọ̀ sí ohun èlò wa!
```

**Challenge:** Add more translations for full page

---

## 🎯 Task 4: Nigerian Recipe App with Favorites

**Task:** Build a recipe app where users can favorite recipes

**Requirements:**
1. Create `FavoritesContext.jsx`
2. State: favorites array
3. Functions: addFavorite, removeFavorite, isFavorite
4. List of 6 Nigerian recipes
5. Heart icon to add/remove favorites
6. Favorites page showing only favorited recipes
7. Use Context to share favorites across pages

**Sample Output:**
```
Recipes:
Jollof Rice ❤️
Egusi Soup 🤍
Suya ❤️

Favorites Page:
Jollof Rice
Suya
```

**Challenge:** Add "Clear All Favorites" button

---


## **Part 8: Review**

### **Key Takeaways**
✅ Context solves prop drilling problem  
✅ Create context with `createContext()`  
✅ Wrap components with Provider  
✅ Access context with `useContext()`  
✅ Create custom hooks for cleaner access  
✅ Can use multiple contexts in one app  
✅ Good for global state (theme, auth, language)  

### **Common Mistakes**
- Using context for everything (overuse)
- Not wrapping app in Provider
- Forgetting to pass value to Provider
- Trying to use context outside Provider
- Creating too many contexts unnecessarily
- Using context for frequently changing data

### **Best Practices**
- Create separate context files in `contexts/` folder
- Create custom hooks (useCart, useTheme, etc.)
- Provide default values to createContext
- Throw error if context used outside provider
- Keep context values focused (one concern per context)
- Don't put everything in context - use local state when possible

### **Context vs Props Quick Guide**

| **Scenario** | **Use** |
|-------------|---------|
| Data used by 1-2 components | Props |
| Data used across entire app | Context |
| Theme/Language settings | Context |
| User authentication | Context |
| Parent to immediate child | Props |
| Frequently changing data | Props/State |
| Global app settings | Context |

---

## **Next Lesson Preview**
Tomorrow we'll learn about **useReducer Hook** - managing complex state logic in React!